# Train Amazon Comprehend Custom Classifier Model

## Note that Amazon Comprehend is currently only supported in a subset of regions: 

* US East (N. Virginia), US East (Ohio), US West (Oregon)
* Canada (Central)
* Europe (London), Europe (Ireland), Europe (Frankfurt)
* Asia Pacific (Mumbai), Asia Pacific (Seoul), Asia Pacific (Tokyo), Asia Pacific (Singapore), Asia Pacific (Sydney)

You can check https://aws.amazon.com/about-aws/global-infrastructure/regional-product-services/ for details and updates. 

## Check dependencies setup

In [2]:
%store -r setup_dependencies_passed
%store -r comprehend_train_s3_uri

try:
    setup_dependencies_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

if not setup_dependencies_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
if not comprehend_train_s3_uri:
    print("****************************************************************************************")
    print("**************** PLEASE RE-RUN THE PREVIOUS DATA PREPARATION NOTEBOOK ******************")
    print("**************** THIS NOTEBOOK WILL NOT RUN PROPERLY ***********************************")
    print("****************************************************************************************")
else:
    print("[OK] Everything is correctly set up")

[OK] Everything is correctly set up


In [3]:
!aws s3 ls $comprehend_train_s3_uri

2022-11-15 18:57:23     411440 amazon_reviews_us_Digital_Software_v1_00_comprehend.csv


## Setup notebook

In [4]:
import csv
import datetime
import json
import time

import boto3
import pandas as pd
import sagemaker
from botocore.exceptions import ClientError

region = boto3.Session().region_name
sm = sagemaker.Session()
bucket = sm.default_bucket()
role = sagemaker.get_execution_role()

from botocore.config import Config

config = Config(retries={"max_attempts": 10, "mode": "adaptive"})

iam = boto3.client("iam", config=config)

### Check if you current regions supports Comprehend

In [5]:
if region in [
    "ap-south-1",
    "eu-west-2",
    "eu-west-1",
    "ap-northeast-2",
    "ap-northeast-1",
    "ca-central-1",
    "ap-southeast-1",
    "ap-southeast-2",
    "eu-central-1",
    "us-east-1",
    "us-east-2",
    "us-west-2",
]:
    print(" [OK] COMPREHEND IS SUPPORTED IN {}".format(region))
    print(" [OK] Please proceed with this notebook.")
else:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print(" [ERROR] COMPREHEND IS NOT YET SUPPORTED IN {}.".format(region))
    print(" [INFO] This is OK. Skip this notebook and continue with the next use case.")
    print(" [INFO] This notebook is not required for the rest of this workshop.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

 [OK] COMPREHEND IS SUPPORTED IN us-east-1
 [OK] Please proceed with this notebook.


In [6]:
comprehend = boto3.client("comprehend")

## See our prepared training data which we use as input for Comprehend

In [7]:
!aws s3 cp $comprehend_train_s3_uri ./tmp/

temp_folder = "tmp"
dataset_csv = "amazon_reviews_us_Digital_Software_v1_00_comprehend.csv"

download: s3://sagemaker-us-east-1-688013747199/data/amazon_reviews_us_Digital_Software_v1_00_comprehend.csv to tmp/amazon_reviews_us_Digital_Software_v1_00_comprehend.csv


In [8]:
df = pd.read_csv("./tmp/amazon_reviews_us_Digital_Software_v1_00_comprehend.csv", header=None)
df.head()

,0,1
0,5,Download was quick and easy. I've only had it ...
1,3,I have been using Quicken Essentials for Mac a...
2,1,Wish there was a zero star rating. I had Offi...
3,1,Those of you experimenting with Windows 8 rele...
4,2,the State download was hard to install it keep...


# Create Data Access Role for Comprehend

## Create Policy

In [9]:
assume_role_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "comprehend.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ],
}

## Create Role and Attach Policies

In [10]:
iam_comprehend_role_name = "DSOAWS_Comprehend"
iam_comprehend_role_description="Curso MLOps Comprehend Role"

In [11]:
try:
    iam_role_comprehend = iam.create_role(
        RoleName=iam_comprehend_role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description=iam_comprehend_role_description,
    )
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        iam_role_comprehend = iam.get_role(RoleName=iam_comprehend_role_name)
        print("Role already exists")
    else:
        print("Unexpected error: %s" % e)

In [12]:
comprehend_s3_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": ["s3:GetObject"],
            "Resource": [f"arn:aws:s3:::{bucket}/*"],
            "Effect": "Allow"
        },
        {
            "Action": ["s3:ListBucket"],
            "Resource": [f"arn:aws:s3:::{bucket}"],
            "Effect": "Allow"
        },
        {
            "Action": ["s3:PutObject"],
            "Resource": [f"arn:aws:s3:::{bucket}/*"],
            "Effect": "Allow"
        },
    ],
}

# Attach Policy to Role

In [13]:
response = iam.put_role_policy(
    RoleName=iam_comprehend_role_name,
    PolicyName="DSOAWS_ComprehendPolicyToS3",
    PolicyDocument=json.dumps(comprehend_s3_policy_doc),
)

print(response)

{'ResponseMetadata': {'RequestId': '6dd1aa17-be56-4db8-85a2-1f9c0c5bad77', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '6dd1aa17-be56-4db8-85a2-1f9c0c5bad77', 'content-type': 'text/xml', 'content-length': '206', 'date': 'Tue, 15 Nov 2022 19:00:46 GMT'}, 'RetryAttempts': 0}}


# Train the Model

In [42]:
prefix = "models"
key = "comprehend/output"

s3_output_job = f"s3://{bucket}/{prefix}/{key}"
print(s3_output_job)

s3://sagemaker-us-east-1-688013747199/models/comprehend/output


In [43]:
iam_role_comprehend_arn = iam_role_comprehend["Role"]["Arn"]

In [44]:
timestamp = str(datetime.datetime.now().strftime("%s"))

comprehend_training_job_name = f"Amazon-Customer-Reviews-Classifier-{timestamp}"

In [45]:
print(comprehend_training_job_name)
print(iam_role_comprehend_arn)
print(comprehend_train_s3_uri)
print(s3_output_job)

Amazon-Customer-Reviews-Classifier-1668541045
arn:aws:iam::688013747199:role/DSOAWS_Comprehend
s3://sagemaker-us-east-1-688013747199/data/amazon_reviews_us_Digital_Software_v1_00_comprehend.csv
s3://sagemaker-us-east-1-688013747199/models/comprehend/output


In [46]:
training_job = comprehend.create_document_classifier(
    DocumentClassifierName=comprehend_training_job_name,
    DataAccessRoleArn=iam_role_comprehend_arn,
    InputDataConfig={"S3Uri": comprehend_train_s3_uri},
    OutputDataConfig={"S3Uri": s3_output_job},
    LanguageCode="en",
)

In [47]:
comprehend_training_job_arn = training_job["DocumentClassifierArn"]

print(comprehend_training_job_arn)

arn:aws:comprehend:us-east-1:688013747199:document-classifier/Amazon-Customer-Reviews-Classifier-1668541045


In [48]:
from IPython.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/comprehend/v2/home?region={}#classifier-details/{}">Comprehend Training Job</a></b>'.format(
            region, comprehend_training_job_arn
        )
    )
)

# This Next Cell Will Take Some Time
# _Please be patient._

In [49]:
help(comprehend.create_document_classifier)

Help on method create_document_classifier in module botocore.client:

create_document_classifier(*args, **kwargs) method of botocore.client.Comprehend instance
    Creates a new document classifier that you can use to categorize documents. To create a classifier, you provide a set of training documents that labeled with the categories that you want to use. After the classifier is trained you can use it to categorize a set of labeled documents into the categories. For more information, see `Document Classification <https://docs.aws.amazon.com/comprehend/latest/dg/how-document-classification.html>`__ in the Comprehend Developer Guide. 
    
    
    
    See also: `AWS API Documentation <https://docs.aws.amazon.com/goto/WebAPI/comprehend-2017-11-27/CreateDocumentClassifier>`_
    
    
    **Request Syntax** 
    ::
    
      response = client.create_document_classifier(
          DocumentClassifierName='string',
          VersionName='string',
          DataAccessRoleArn='string',
    

In [51]:
import time

max_time = time.time() + 3 * 60 * 60  # 3 hours
while time.time() < max_time:
    describe_custom_classifier = comprehend.describe_document_classifier(
        DocumentClassifierArn=comprehend_training_job_arn
    )
    status = describe_custom_classifier["DocumentClassifierProperties"]["Status"]
    print(f"Custom classifier: {status}")

    if status == "TRAINED" or status == "IN_ERROR":
        print("")
        print(f"Status {status}")
        print("")
        print(describe_custom_classifier["DocumentClassifierProperties"])
        break

    time.sleep(10)

Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: T

# _Please Wait Until the ^^ Classifier ^^ is Trained Above._

# [INFO] _Feel free to continue to the next workshop section while this notebook is running._

# Show Results of the Classifier

In [52]:
print(describe_custom_classifier["DocumentClassifierProperties"])

{'DocumentClassifierArn': 'arn:aws:comprehend:us-east-1:688013747199:document-classifier/Amazon-Customer-Reviews-Classifier-1668541045', 'LanguageCode': 'en', 'Status': 'TRAINED', 'SubmitTime': datetime.datetime(2022, 11, 15, 19, 37, 27, 660000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2022, 11, 15, 20, 19, 58, 944000, tzinfo=tzlocal()), 'TrainingStartTime': datetime.datetime(2022, 11, 15, 19, 43, 45, 141000, tzinfo=tzlocal()), 'TrainingEndTime': datetime.datetime(2022, 11, 15, 20, 19, 8, 682000, tzinfo=tzlocal()), 'InputDataConfig': {'DataFormat': 'COMPREHEND_CSV', 'S3Uri': 's3://sagemaker-us-east-1-688013747199/data/amazon_reviews_us_Digital_Software_v1_00_comprehend.csv'}, 'OutputDataConfig': {'S3Uri': 's3://sagemaker-us-east-1-688013747199/models/comprehend/output/688013747199-CLR-9bf5224d1052eb06da7255a5f2a7f287/output/output.tar.gz'}, 'ClassifierMetadata': {'NumberOfLabels': 5, 'NumberOfTrainedDocuments': 810, 'NumberOfTestDocuments': 90, 'EvaluationMetrics': {'Accuracy': 

In [53]:
model_arn = describe_custom_classifier["DocumentClassifierProperties"]["DocumentClassifierArn"]
print(model_arn)

arn:aws:comprehend:us-east-1:688013747199:document-classifier/Amazon-Customer-Reviews-Classifier-1668541045


In [54]:
import os

# Retrieve the S3URI from the model output and create jobkey variable.
job_output = describe_custom_classifier["DocumentClassifierProperties"]["OutputDataConfig"]["S3Uri"]
print(job_output)

path_prefix = "s3://{}/".format(bucket)

job_key = os.path.relpath(job_output, path_prefix)

print(job_key)

s3://sagemaker-us-east-1-688013747199/models/comprehend/output/688013747199-CLR-9bf5224d1052eb06da7255a5f2a7f287/output/output.tar.gz
models/comprehend/output/688013747199-CLR-9bf5224d1052eb06da7255a5f2a7f287/output/output.tar.gz


# Download Model Artifacts including Training Metrics

In [57]:
s3 = boto3.resource("s3")
s3.Bucket(bucket).download_file(job_key, f"{temp_folder}/output.tar.gz")

#s3 = boto3.client("s3")

#s3.download_file(
#    Bucket=bucket,
#    Key=job_key,
#    Filename=f"{temp_folder}/output.tar.gz"
#)

In [60]:
# Unpack the gzip file
!tar xvzf ./tmp/output.tar.gz

output/
output/confusion_matrix.json


In [61]:
import json

with open("./output/confusion_matrix.json") as json_file:
    data = json.load(json_file)
print(json.dumps(data, indent=2, default=str))

{
  "confusion_matrix": [
    [
      7,
      4,
      2,
      5,
      0
    ],
    [
      1,
      6,
      7,
      3,
      1
    ],
    [
      0,
      6,
      3,
      6,
      3
    ],
    [
      1,
      5,
      3,
      7,
      2
    ],
    [
      1,
      1,
      0,
      2,
      14
    ]
  ],
  "labels": [
    "1",
    "2",
    "3",
    "4",
    "5"
  ],
  "type": "multi_class",
  "all_labels": [
    "1",
    "2",
    "3",
    "4",
    "5"
  ]
}


In [62]:
from IPython.display import HTML, display
import tabulate

table = [
    ["", "1", "2", "3", "4", "5", "(Predicted)"],
    [
        "1",
        data["confusion_matrix"][0][0],
        data["confusion_matrix"][0][1],
        data["confusion_matrix"][0][2],
        data["confusion_matrix"][0][3],
        data["confusion_matrix"][0][4],
    ],
    [
        "2",
        data["confusion_matrix"][1][0],
        data["confusion_matrix"][1][1],
        data["confusion_matrix"][1][2],
        data["confusion_matrix"][1][3],
        data["confusion_matrix"][1][4],
    ],
    [
        "3",
        data["confusion_matrix"][2][0],
        data["confusion_matrix"][2][1],
        data["confusion_matrix"][2][2],
        data["confusion_matrix"][2][3],
        data["confusion_matrix"][2][4],
    ],
    [
        "4",
        data["confusion_matrix"][3][0],
        data["confusion_matrix"][3][1],
        data["confusion_matrix"][3][2],
        data["confusion_matrix"][3][3],
        data["confusion_matrix"][3][4],
    ],
    [
        "5",
        data["confusion_matrix"][4][0],
        data["confusion_matrix"][4][1],
        data["confusion_matrix"][4][2],
        data["confusion_matrix"][4][3],
        data["confusion_matrix"][4][4],
    ],
    ["(Actual)"],
]
display(HTML(tabulate.tabulate(table, tablefmt="html")))

,1,2,3,4,5,(Predicted)
1,7,4,2,5,0,
2,1,6,7,3,1,
3,0,6,3,6,3,
4,1,5,3,7,2,
5,1,1,0,2,14,
(Actual),,,,,,


# Deploy Endpoint

In [63]:
from time import gmtime, strftime, sleep

timestamp_suffix = strftime("%d-%H-%M-%S", gmtime())

comprehend_endpoint_name = "comprehend-inference-ep-" + timestamp_suffix

inference_endpoint_response = comprehend.create_endpoint(
    EndpointName=comprehend_endpoint_name, ModelArn=model_arn, DesiredInferenceUnits=1
)

In [64]:
comprehend_endpoint_arn = inference_endpoint_response["EndpointArn"]
print(comprehend_endpoint_arn)

arn:aws:comprehend:us-east-1:688013747199:document-classifier-endpoint/comprehend-inference-ep-15-20-28-20


# Pass Variables to the Next Notebook(s)

In [65]:
%store comprehend_training_job_arn
%store comprehend_endpoint_arn

Stored 'comprehend_training_job_arn' (str)
Stored 'comprehend_endpoint_arn' (str)


# Release Resources

In [66]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>

In [67]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>